In [ ]:
# imports
import os
import imp
import numpy as np
import pandas as pd
import theano
import lasagne
import loading
from training import *
from network import *
from architectures import *
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import bayes_mvs as bmvs

%matplotlib inline

# aliases
L = lasagne.layers
nl = lasagne.nonlinearities
T = theano.tensor

# data loading
headdir = os.path.expanduser('~/Google Drive/Bas Zahy Gianni - Games')
paramsdir = os.path.join(headdir, 'Analysis/0_hvh/Params/nnets/temp')
datafile = os.path.join(headdir, 'Data/0_hvh/Clean/_summaries/model_input_with_groups.csv')
resultsdir = os.path.join(headdir, 'Analysis/0_hvh/Loglik/nnets')
data = loading.default_loader(datafile)

## Basic Prototyping

In [ ]:
# prototype = default_convnet

def prototype(input_var=None):
    input_shape = (None, 2, 4, 9)
    FixLayer = make_FixLayer(input_var)

    input_layer = L.InputLayer(shape=input_shape, input_var=input_var)
    network = L.Conv2DLayer(
        input_layer, num_filters=32, filter_size=(4,4), pad='full',
        nonlinearity=nl.identity
    )

    network = L.ParametricRectifierLayer(network, shared_axes='auto') # default: auto
    network = L.FeaturePoolLayer(network, pool_function=T.sum, pool_size=2) # default: T.sum, 2
    network = L.DropoutLayer(network, p=.75) # default: .75
    network = L.DenseLayer(
        network, num_units=36,
        nonlinearity=nl.very_leaky_rectify, W=lasagne.init.HeUniform(gain='relu')
    )
    network = FixLayer(network)
    network = L.NonlinearityLayer(network, nonlinearity=nl.softmax)
    network = FixLayer(network)
    network = ReNormLayer(network)

    return network

trainer = DefaultTrainer(stopthresh=100) # default: 125
net_list = trainer.train_all(architecture=prototype, data=data, seed=985227)

for i, n in enumerate(net_list):
    n.save_params(os.path.join(paramsdir, '{} split agg fit'.format(i)))

## Subject finetuning prototyping

In [ ]:
def prototype(input_var=None):
    input_shape = (None, 2, 4, 9)
    FixLayer = make_FixLayer(input_var)

    input_layer = L.InputLayer(shape=input_shape, input_var=input_var)
    network = L.Conv2DLayer(
        input_layer, num_filters=32, filter_size=(4,4), pad='full',
        nonlinearity=nl.identity
    )

    network = L.ParametricRectifierLayer(network, shared_axes='auto') # default: auto
    network = L.FeaturePoolLayer(network, pool_function=T.sum, pool_size=4) # default: T.sum, 2
    network = L.DropoutLayer(network, p=.875) # default: .75
    network = L.DenseLayer(
        network, num_units=36,
        nonlinearity=nl.leaky_rectify, W=lasagne.init.HeUniform(gain='relu')
    )
    network = FixLayer(network)
    network = L.NonlinearityLayer(network, nonlinearity=nl.softmax)
    network = FixLayer(network)
    network = ReNormLayer(network)

    return network

class FineTuner(DefaultTrainer):
    """
    Trainer to fine tune networks to individual subjects

    Consider moving freeze, param set functions properly into Network object
    Abstracting split functions and augment in DefaultTrainer would be good too
    """

    def train_all(self, architecture, data, split=0, seed=None, startparams=None, freeze=True, save_params=False):
        if seed:
            np.random.seed(seed)

        D, groups, Xs, ys, Ss = data
        num_splits = len(Xs)
        r = np.tile(np.arange(num_splits), [num_splits, 1])
        r = (r + r.T) % num_splits

        starttime = time.time()
        net = Network(architecture)
        if startparams:
            _layers = L.get_all_layers(net.net)[1:3]
            L.set_all_param_values(net.net, startparams)#(_layers, startparams[0:3])
            convlayer, prelulayer = _layers
            if freeze:
                convlayer.params[convlayer.W].remove('trainable')
                convlayer.params[convlayer.b].remove('trainable')
                prelulayer.params[prelulayer.alpha].remove('trainable')

        train_idxs = r[split, :3]
        val_idxs = r[split, 3:4]
        test_idxs = r[split, 4:]

        X, y, S = [np.concatenate(np.array(Z)[train_idxs]) for Z in [Xs, ys, Ss]]
        Xv, yv, Sv = [np.concatenate(np.array(Z)[val_idxs]) for Z in [Xs, ys, Ss]]
        Xt, yt, St = [np.concatenate(np.array(Z)[test_idxs]) for Z in [Xs, ys, Ss]]
#         print(X.shape[0])
        if X.shape[0] > 75:
            X, y = augment((X, y))
            S = np.concatenate([S, S, S, S])
            self.train(net, training_data=(X, y), validation_data=(Xv, yv))
        self.test(net, testing_data=(Xt, yt))
        time_elapsed = time.time() - starttime

        return net


In [ ]:
# TODO: save traces
# TODO: keep traces from one net to another?? or save separately and append...

# set fixed params with easy-access dicts, because we don't hate ourselves
#   also because we don't hate ourselves, maybe name these more clearly soon?

trainer_settings = {
    'stopthresh':3,
#     'updates': lasagne.updates.sgd,
    'update_args':{'learning_rate':.001}
}

trainer_args = {
    'architecture':prototype,
    'seed':985227,
}

subject_Xs = data[2]
subject_ys = data[3]
subject_Ss = data[4]
results = []
for i, net in enumerate(net_list):
    result = []
    PARAMS = L.get_all_param_values(net.net)
    
    for subno in range(40):
        S0 = [np.where(s==subno)[0] for s in subject_Ss]
        X0 = [subject_Xs[i][s] for i, s in enumerate(S0)]
        y0 = [subject_ys[i][s] for i, s in enumerate(S0)]
        sub_data = (data[0], data[1], X0, y0, S0)
        print('\nMODEL {}, SUBJECT {}, NMOVES {} per split\n'.format(i, subno, len(S0[0])))

        batchsize = len(S0[0])//2
        sub_trainer = FineTuner(batchsize=batchsize, **trainer_settings)
        sub_net = sub_trainer.train_all(data=sub_data, startparams=PARAMS, split=i, **trainer_args)
        result.append(sub_net.test_err)
        sub_net.save_params(os.path.join(paramsdir, '{} split {} sub tune fit'.format(i, subno))) # something is broken here
        # add trace saving here
        
        
    results.append(result)

res = np.array(results).T
# np.savetxt(os.path.join(resultsdir, 'nlls.txt'), res, fmt='%.18f', delimiter=',')


In [ ]:
res.mean()

In [ ]:
# generate boards file

# TEMPORARY subclass - merge these functions into main network, since they are useful!
class TestNetwork(Network):
    def __init__(self, architecture):
        super(TestNetwork, self).__init__(architecture)
        self.itemized_loss = lasagne.objectives.categorical_crossentropy(self.test_prediction, self.target_var)
        self.itemized_test_fn = theano.function([self.input_var, self.target_var], self.itemized_loss)
        
    def load_params(self, paramsfile):
        with np.load(paramsfile) as loaded:
            params_list = [(i[0], i[1]) for i in loaded.items()]
            params_list.sort()
            L.set_all_param_values(self.net, [i[1] for i in params_list])
            

D = data[0].copy()
D2 = data[0].copy()
D['group'] = D['group']-1
D2['group'] = D2['group']-1
O = pd.DataFrame(index=D.index, columns=list(range(36)))
Xx, yy, Ss, Gg, Npnp = loading.unpack_data(D)
for _g in D['group'].unique():
    print('Split group: {}, Model idx: {}\n'.format(_g, (_g+1)%5))
    for _s in D['subject'].unique():
        paramsfile = os.path.join(paramsdir, '{} split {} sub tune fit.npz'.format((_g+1)%5, _s))
        locs = (D['subject']==_s)&(D['group']==_g)
        idx = D.loc[locs, :].index.values
        test_net = TestNetwork(prototype)
        test_net.load_params(paramsfile)
        
        _x = Xx[idx, :, :, :]
        _y = yy[idx]
        print(_s, test_net.test_fn(_x, _y))

        D.loc[idx, 'nll'] = test_net.itemized_test_fn(_x, _y)
        O.loc[idx, list(range(36))] = test_net.output_fn(_x)
        O.loc[idx, 'prediction'] = O.loc[idx, list(range(36))].values.argmax(axis=1)
    paramsfile = os.path.join(paramsdir, '{} split agg fit.npz'.format((_g+1)%5))
    idx = D2.loc[D2['group']==_g, :].index.values
    test_net = TestNetwork(prototype)
    test_net.load_params(paramsfile)
    
    _x = Xx[idx, :, :, :]
    _y = yy[idx]
    D2.loc[idx, 'nll'] = test_net.itemized_test_fn(_x, _y)

D = D.join(O)
D['correct'] = (D['zet']==D['prediction']).astype(int)
D.head()

In [ ]:
p = D.pivot_table(index='subject', values=['nll']).values.flatten()
p2 = D2.pivot_table(index='subject', values=['nll']).values.flatten()
diff = p - p2
lendiff = D.pivot_table(index='subject', values=['nll'], aggfunc=len).values.flatten()
order = p2.argsort()
from scipy.stats import linregress
lr = linregress(lendiff, diff)
print(lr)

fig, axes = plt.subplots(2, 2, figsize=(10,8))
ax = axes[0, 0]
ax.plot(lendiff, diff, color='black', linestyle='none', marker='o')
ax.text(25, -.18, s='corr = {:.5f}'.format(lr.slope))
plt.setp(axes[0, 0], xlabel='N Moves in data', ylabel='Delta NLL')
ax = axes[0, 1]
ax.plot(lendiff, p2, color='black', linestyle='none', marker='o')
plt.setp(ax, xlabel='N Moves in data', ylabel='NLL')
ax = axes[1, 1]
ax.plot(lendiff, p, color='red', linestyle='none', marker='o')
plt.setp(ax, xlabel='N Moves in data', ylabel='NLL')

axes[1, 0].plot(np.arange(40), p2[order], color='black', linestyle='none', marker='o', label='No tune')
axes[1, 0].plot(np.arange(40), p[order], color='red', linestyle='none', marker='o', label='Fine tune')
axes[1, 0].legend(loc=0)
plt.setp(axes[1, 0], xlabel='Ranked subject', ylabel='Nll')
sns.despine()
fig.savefig(os.path.join(resultsdir, 'overfitting viz exclude n lt 125.png'), format='png')

In [ ]:
D2['nll'].mean()

In [ ]:
p2.mean()

In [ ]:
diff.shape

In [ ]:
D.to_csv(os.path.join(resultsdir, '(POSSIBLE BUG - NLL MISMATCH) appended data.csv'))

In [ ]:
mos = [bmvs(res[i, :], alpha=.95) for i in range(res.shape[0])]
means = np.array([m[0][0] for m in mos])
lbs = np.array([m[0][1][0] for m in mos])
ubs = np.array([m[0][1][1] for m in mos])
idx = np.argsort(means)
plt.fill_between(x=np.arange(40), y1=lbs[idx], y2=ubs[idx], alpha=.2)
plt.plot(means[idx])
plt.setp(plt.gca(), xlabel='Ranked Subject', ylabel='NLL')
sns.despine()

In [ ]:
plt.hist(res.flatten(), bins = np.arange(1, 4, 1/8))
plt.setp(plt.gca(), xlabel='NLL', ylabel='# Fits')
sns.despine()

In [ ]:
bmvs(np.array(res).flatten())

## Autoencoder prototyping

In [ ]:
def autoencoder_prototype(input_var=None):
    input_shape = (None, 2, 4, 9)
    inpuL.InputLayer = L.InputLayer(shape=input_shape, input_var=input_var)
    network = L.COnv2DLayer(
        input_layer, num_filters=32, filter_size=(4, 4), pad='full',
        nonlinearity=nl.identity
    )
    L.ParametricRectifierLayerrLayeshared_axes=Rectifier(network, shL.FeaturePoolLayerl')
    network = L.FeaturePoolLayer(network, pool_functioL.DropoutLayerm, pool_size=2)
    network = L.DropoutLayer(network, p=.75)
    network = L.DenseLayer(
        network, num_units=72,
        nonlinearity=nl.very_leaky_rectify, W=lasagne.init.HeUniform(gain='relu')
    )
    network = L.ReshapeLayer(
        network, shape=(-1, 2, 4, 9)
    )
    return network

In [ ]:
class Autoencoder(Network):
    """
    Overwrites loss functions for training an autoencoder instead
    of softmax output classifier
    """
    def objectives(self):
        """Must define: loss, test_loss, test_acc"""
        self.loss = lasagne.objectives.squared_error(self.prediction, self.target_var)
        self.loss = self.loss.mean()
        self.test_loss = lasagne.objectives.squared_error(self.test_prediction, self.target_var)
        self.test_loss = self.test_loss.mean()
        self.test_acc = T.mean(T.abs(self.test_prediction - self.target_var))
        return None

# class AutoTrainer(DefaultTrainer):
#     """
#     Overwrites training functions for training an autoencoder instead
#     of softmax output classifier
#     """
#     pass

# Can actually pass data to existing trainer by replacing ys with a copy of Xs!

# class LadderNet(Network):
#     """
#     Loads conv layer params from autoencoder save and sets them on current classifier
#     Needs overwrite of self.build() that loads conv2d params, then sets them untrainable.
#     """
#     def __init__(self, architecture, paramsfile):
#         self.paramsfile = paramsfile
#         super(LadderNet, self).__init__(architecture)
    
#     def build(self):
#         super(LadderNet, self).build()
        
#         # Finish this soon!
#         return None
    
#     def load_conv_params(self):
#         with np.load(paramsfile) as loaded:
# #             L.